In [3]:
# Add PyDatAnalysis to path
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/Users/owensheekey/Documents/Research/PyDatAnalysis")

export_path = 'Exports/'

In [4]:
from __future__ import annotations
from progressbar import progressbar
from src.DatObject.Make_Dat import get_dat, get_dats
import src.UsefulFunctions as U
from src.DataStandardize.ExpSpecific.Feb21 import Feb21Exp2HDF, Feb21ExpConfig
from src.DataStandardize.ExpConfig import ExpConfigGroupDatAttribute, ExpConfigBase
import multiprocessing as mp
import plotly.graph_objs as go
import numpy as np
import lmfit as lm
from typing import TYPE_CHECKING, Iterable, Optional
from src.DatObject.Attributes.Transition import i_sense_digamma, i_sense, i_sense_digamma_quad
from src.UsefulFunctions import edit_params
from src.DatObject.Attributes.SquareEntropy import square_wave_time_array, integrate_entropy
import logging
logger = logging.getLogger(__name__)
import src.UsefulFunctions as U
from src.CoreUtil import decimate

In [5]:
def narrow_fit(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    out = dat.SquareEntropy.get_Outputs(existing_only=True)
    x = np.copy(out.x)
    y = np.copy(out.averaged)
    y = np.mean(y[(0, 2), :], axis=0)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.SquareEntropy.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

In [6]:
def narrow_fit_trans_only(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    x = np.copy(dat.Transition.avg_x)
    y = np.copy(dat.Transition.avg_data)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.Transition.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

In [7]:
def do_calc(datnum, overwrite=True):
    """Just a function which can be passed to a process pool for faster calculation"""
    save_name = 'SPS.01'

    dat = get_dat(datnum)

    setpoints = [0.01, None]

    # Get other inputs
    setpoint_times = square_wave_time_array(dat.SquareEntropy.square_awg)
    sp_start, sp_fin = [U.get_data_index(setpoint_times, sp) for sp in setpoints]
    logger.debug(f'Setpoint times: {setpoints}, Setpoint indexs: {sp_start, sp_fin}')

    # Run Fits
    pp = dat.SquareEntropy.get_ProcessParams(name=None,  # Load default and modify from there
                                             setpoint_start=sp_start, setpoint_fin=sp_fin,
                                             transition_fit_func=i_sense,
                                             save_name=save_name)
    out = dat.SquareEntropy.get_Outputs(name=save_name, inputs=None, process_params=pp, overwrite=overwrite)
    dat.Entropy.get_fit(which='avg', name=save_name, data=out.average_entropy_signal, x=out.x, check_exists=False,
                        overwrite=overwrite)
    [dat.Entropy.get_fit(which='row', row=i, name=save_name,
                         data=row, x=out.x, check_exists=False,
                         overwrite=overwrite) for i, row in enumerate(out.entropy_signal)]
    return out

In [8]:
def get_deltaT(dat):
    """Returns deltaT of a given dat in mV"""
    ho1 = dat.AWG.max(0)  # 'HO1/10M' gives nA * 10
    t = dat.Logs.temps.mc

    # Datnums to search through (only thing that should be changed)
    datnums = set(range(2143, 2156))
    # datnums = set()
    # for j in range(5):
    #     datnums = datnums.union(set(range(28 * j + 1312, 28 * j + 1312 + 4 * 7 + 1)) - set([28 * j + 1312 + 4 * i for i in range(8)]))
    # datnums = list(datnums)

    dats = get_dats(datnums)

    dats = [d for d in dats if np.isclose(d.Logs.temps.mc, dat.Logs.temps.mc, rtol=0.1)]  # Get all dats where MC temp is within 10%
    bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])

    indp = np.argmin(abs(bias_lookup - ho1))
    indm = np.argmin(abs(bias_lookup + ho1))
    theta_z = np.nanmean([d.Transition.avg_fit.best_values.theta for d in dats if d.Logs.fds['HO1/10M'] == 0])

    # temp_lookup = np.array([d.Logs.temps.mc for d in dats])
    # bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])
    #
    # indp = np.argmin(temp_lookup - t + bias_lookup - ho1)
    # indm = np.argmin(temp_lookup - t + bias_lookup + ho1)
    # indz = np.argmin(temp_lookup - t + bias_lookup)

    theta_p = dats[indp].Transition.avg_fit.best_values.theta
    theta_m = dats[indm].Transition.avg_fit.best_values.theta
    # theta_z = dats[indz].Transition.avg_fit.best_values.theta
    return (theta_p + theta_m) / 2 - theta_z

In [84]:
def get_simple_avg(dat):
    return dat.Data.get_data('Experiment Copy/x_array'), np.average(dat.Data.get_data('Experiment Copy/cscurrent_2d'), axis=0)

In [119]:
# datnums = set(range(1869, 1919)) - set(range(1870, 1919, 2))
# transdatnums = set(range(1869, 1919)) - set(range(1869, 1919, 2))

# datnums = np.sort(list(set(range(2089, 2095)) - set(range(2090, 2095, 2))))
# transdatnums = np.sort(list(set(range(2089, 2095)) - set(range(2089, 2095, 2))))
# datnums = np.sort(list(set(range(1778, 1795))))

# datnums = np.sort(list(set(range(2156, 2162)) - set(range(2157, 2162, 2))))
# transdatnums = np.sort(list(set(range(2156, 2162)) - set(range(2156, 2162, 2))))

datnums = [2164,2167, 2170, 2176, 2160, 2131, 2178, 2180, 2182]
transdatnums = [2165,2168, 2171, 2177, 2161, 2132, 2179, 2181, 2183]

# datnums = np.sort(list(set(range(2095, 2143)) - set(range(2096, 2143, 2))))
# transdatnums = np.sort(list(set(range(2095, 2143)) - set(range(2095, 2143, 2))))
end = 2806
datnums = np.sort(list(set(range(2659, end)) - set(range(2660, end, 2)) - set([2793])))
transdatnums = np.sort(list(set(range(2659, end)) - set(range(2659, end, 2)) - set([2794])))
# datnums = np.array([2699, 2719, 2731, 2737, 2741, 2757])
# transdatnums = np.add(datnums, 1)
#datnums = np.sort(list(set(range(3066, 3255+1))))
datnums = np.sort(list(set(range(3451, 3551+1))))
#datnums = np.sort(list(set(range(3085, 3244+1)) | set(range(3430, 3450+1))))
#datnums = np.sort(list(set(range(3551, 3652+1))))
datnums = np.sort(list(set(range(3671, 3698+1))))
datnums = np.sort(list(set(range(3769, 3890+1))))
transdatnums = np.sort(list((set(range(3898, 3902+1)) | set([3904,3907,3909, 3910, 3911, 3912, 3913, 3914, 3915, 3916, 3917, 3918])) - set([3899])))
transdatnums = np.sort(list(set(range(3953, 3961+1))))
transdatnums = np.sort(list(set(range(4220, 4228+1))))
transdatnums = np.sort(list(set(range(4322, 4330+1))))
transdatnums = np.sort(list(set(range(4333, 4340+1))))
transdatnums = np.sort(list(set(range(5534, 5542+1)) - set([5539])))
transdatnums = np.sort(list(set(range(5543, 5558+1))))

In [120]:
transdatnums

array([5543, 5544, 5545, 5546, 5547, 5548, 5549, 5550, 5551, 5552, 5553,
       5554, 5555, 5556, 5557, 5558])

In [121]:
#dats = get_dats(list(datnums), overwrite=False)
transdats = get_dats(list(transdatnums), overwrite=False) #np.mean(dat.Data.sweepgates_x[1][1:])
escs = [dat.Logs.fds["ESC"] for dat in transdats]
iss = [dat.Logs.bds["IS"] for dat in transdats]

In [122]:
# Θ = np.average([dat.SquareEntropy.get_fit(which='avg',which_fit='transition', transition_part='cold', check_exists=False).best_values.theta for dat in progressbar(dats)])
fit = transdats[0].Transition.get_fit(which='avg', check_exists=False)
Θ = 2.2
params = fit.params
params.add('g', value=0, vary=True, min=-50, max=1000)
new_pars = edit_params(params, param_name='theta', value=Θ, vary=False)

In [114]:
'''USING SIMPLE FITS'''

fits = []
for dat in progressbar(transdats):
    x, y = get_simple_avg(dat)
    fits.append(dat.Transition.get_fit(data=y, x=x, check_exists=False, initial_params=new_pars, 
                fit_func=i_sense_digamma))
amp_digamma_ = [f.best_values.amp for f in fits]
g_digamma_ = [f.best_values.g for f in fits]
theta_digamma_ = [f.best_values.theta for f in fits]

100% (8 of 8) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [123]:
'''NOT USING SIMPLE FITS'''

fits = []
for dat in progressbar(transdats):
    fits.append(dat.Transition.get_fit(which='avg', check_exists=False, initial_params=new_pars, 
                fit_func=i_sense_digamma))
amp_digamma_ = [f.best_values.amp for f in fits]
g_digamma_ = [f.best_values.g for f in fits]
theta_digamma_ = [f.best_values.theta for f in fits]

100% (16 of 16) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [124]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=iss, y=theta_digamma_, text=transdatnums, name="Transition fits"))

fig.update_layout(xaxis_title='IS /mV', yaxis_title='Theta /mV',
                      title=f'Dats {transdats[0].datnum} - {transdats[-1].datnum}')
fig.show(renderer="chrome")

In [29]:
fig.write_html(f'Exports/Thetas_dats_{transdats[0].datnum}_{transdats[-1].datnum}.html')

In [125]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=escs, y=np.divide(g_digamma_,Θ), text=transdatnums, name="Transition fits"))

fig.update_layout(xaxis_title='ESC /mV', yaxis_title='g/t /arb',
                      title=f'Dats {transdats[0].datnum} - {transdats[-1].datnum}')
fig.show(renderer="chrome")

In [126]:
fig.write_html(f'Exports/g_over_t_dats{transdats[0].datnum}_{transdats[-1].datnum}.html')

In [118]:
fig = go.Figure()
for i, dat in progressbar(enumerate(transdats)):
    x, y = get_simple_avg(dat)
    xfit = np.linspace(-500,500,1001)
    fit = fits[i]
    yfit = np.subtract(fit.eval_fit(xfit), np.array(fit.best_values.lin*xfit))
    fig.add_trace(go.Scatter(mode='markers', x=x[::10], y=y[::10] - fit.best_values.lin*x[::10], name=f'{transdatnums[i]}d'))
    fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{transdatnums[i]}f', marker_color='grey'))
fig.update_layout(xaxis_title='IP1*200 /mV', yaxis_title='Current /nA',
                      title=f'Dat {transdats[0].datnum} - {transdats[-1].datnum}')
fig.show(renderer="browser")

| |#                                                  | 7 Elapsed Time: 0:00:00


In [42]:
dir(transdats[0].Transition)

['AUTO_BIN_SIZE',
 'DEFAULT_DATA_NAME',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_avg_data',
 '_avg_data_std',
 '_avg_fit',
 '_avg_x',
 '_calculate_fit',
 '_check_default_group_attrs',
 '_create_group',
 '_generate_fit_path',
 '_generate_fit_saved_name',
 '_get_FitPaths',
 '_get_data',
 '_get_default_data',
 '_get_default_x',
 '_get_fit_from_path',
 '_get_fit_parent_group_name',
 '_get_fit_path_from_fit_id',
 '_get_fit_path_from_name',
 '_get_initialized_state',
 '_get_private_key',
 '_initialize',
 '_make_avg_data',
 '_row_fits',
 '_run_all_row_fits',
 '_save_fit',
 '_set_attr',
 '_set_dat

In [1356]:
fig.write_html(f'Exports/Cold_hot_transitions_plus_fit_dats{dats[0].datnum}_{dats[-1].datnum}.html')

In [160]:
ampl = [0.425 for dat in dats]
deltaT = [0.93 for dat in dats]
for i, dat in enumerate(dats):
    dat.Entropy.set_integration_info(dT=deltaT[i], amp=ampl[i], overwrite=True)

NameError: name 'dats' is not defined

In [473]:
fig = go.Figure()
int_ents = []
int_ents_peaks = []
for i in range(len(dats)):
    width = dat.Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).best_values.theta*10
    center = dat.Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).best_values.mid
    indstart = np.argmin(np.abs(np.subtract(out[i].x, center-width)))
    indend = np.argmin(np.abs(np.subtract(out[i].x, center+width)))
    
    int_ent = integrate_entropy(out[i].average_entropy_signal, dats[i].Entropy.integration_info.sf)
    int_ents_peaks.append(max(int_ent))
    int_ents.append(int_ent[-1])
    fig.add_trace(go.Scatter(mode='markers', 
                             x=np.subtract(out[i].x, mids_digamma_[i]), 
                             y=int_ent,
                             text=f'Int Ent = {int_ents[i]:.2f}',
                             name= f'dat{dats[i].datnum}, IP*200:{dats[i].Logs.fds["IP1*200"]}'))

fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer='browser')
print(width)

25.682913401502972


In [405]:
fig.write_html(f'Exports/IntEntropy_03_22_dats{dats[0].datnum}_{dats[-1].datnum}.html')

In [474]:
ents = [dat.Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).best_values.dS for dat in dats]
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers', 
                         x=ips, y=int_ents, 
                         text=datnums, 
                         name="Int",
                         marker=dict(color='LightSkyBlue',
                                     size=10,
                                     line=dict(color='DarkSlateGrey',
                                               width=2))))
fig.add_trace(go.Scatter(mode='markers', 
                         x=ips, y=ents, 
                         text=datnums, 
                         name="Fit",
                         marker=dict(color='Brown',
                                     size=10,
                                     line=dict(color='DarkSlateGrey',
                                               width=2))))
fig.add_trace(go.Scatter(mode='markers', 
                         x=ips, y=int_ents_peaks, 
                         text=datnums, 
                         name="Integrated Peak",
                         marker=dict(color='Green',
                                     size=10,
                                     line=dict(color='DarkSlateGrey',
                                               width=2))))


fig.update_layout(xaxis_title='IP1/200 /mV', yaxis_title='Entropy /kb',
                      title=f'Integrated -- Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [324]:
fig.write_html(f'Exports/IntEntropyvalues_03_21_dats{datnums[0]}_{datnums[-1]}.html')

In [454]:
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers', 
                         x=ips, y=np.subtract(int_ents_peaks,int_ents), 
                         text=datnums, 
                         name="Peak-Int",
                         marker=dict(color='Green',
                                     size=10,
                                     line=dict(color='DarkSlateGrey',
                                               width=2))))

fig.update_layout(xaxis_title='IP1/200 /mV', yaxis_title='Entropy /kb',
                      title=f'Integrated -- Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [439]:
fig = go.Figure()
for i in range(len(dats)):
    enty = out[i].average_entropy_signal
    entx = out[i].x
    fitent = dats[i].Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).eval_fit(entx)
    ent = dats[i].Entropy.get_fit(which='avg', name="SPS.01", check_exists=True).best_values.dS
    fig.add_trace(go.Scatter(mode='markers', 
                             x=entx,
                             y=enty,
                             name= f'dat{dats[i].datnum}, IP*200:{dats[i].Logs.fds["IP1*200"]}'))
    
    fig.add_trace(go.Scatter(mode='lines', x=entx, y=fitent, name=f'{datnums[i]}f, {ent}', marker_color='grey'))

fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer='browser')

In [409]:
fig.write_html(f'Exports/FitEntropy_03_22_dats{datnums[0]}_{datnums[-1]}.html')

In [400]:
centers = [dat.SquareEntropy.get_fit(data=dat.Data.get_data('Experiment Copy/cscurrent_2d')[0],
                                     name="Centers",
                                     x=dat.Data.get_data('Experiment Copy/x_array'),
                                     which_fit='transition', 
                                     check_exists=False).best_values.mid
for dat in progressbar(dats)]

100% (34 of 34) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


In [411]:
occ_data = get_dat(3734)
data = occ_data.Data.get_data('Experiment Copy/cscurrent_2d')
x=occ_data.Data.get_data('Experiment Copy/x_array')
y=occ_data.Data.get_data('Experiment Copy/y_array')
accs = [dat.Logs.fds["ACC*100"] for dat in dats]

In [412]:
fig = go.Figure()
ips_other = np.array(ips)
fig.add_trace(go.Heatmap(
                    z=np.diff(data), x=x[:-1], y=y))
fig.add_trace(go.Scatter(mode='markers+lines', 
                         x=centers, y=ips_other, 
                         text=[dat.datnum for dat in dats],
                         marker_color="White"))
fig.update_layout(xaxis_title='ACC*100 /mV', yaxis_title='IP1*200',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}, dat{occ_data.datnum}')
fig.show(renderer='browser')

In [413]:
fig.write_html(f'Exports/Along_occ_03_21_{dats[0].datnum}_{dats[-1].datnum}.html')

In [218]:
arr = np.array([-65, -60, -55, -50, -45, -40, -30])

In [219]:
a = arr+50

In [221]:
a*-2.5 + 157.78

array([195.28, 182.78, 170.28, 157.78, 145.28, 132.78, 107.78])

In [238]:
arr2 = np.linspace(-520,-440,9)

In [239]:
arr2 = arr2 + 530

In [241]:
arr2*-0.75 + -518.8

array([-526.3, -533.8, -541.3, -548.8, -556.3, -563.8, -571.3, -578.8,
       -586.3])

In [246]:
arr = np.array([200.28, 187.78, 175.28, 162.78, 150.28, 137.78, 112.78]) + 7

In [247]:
arr

array([207.28, 194.78, 182.28, 169.78, 157.28, 144.78, 119.78])